In [0]:
# Importando módulo de seguimiento
import sys
import importlib
sys.path.append('/Workspace/Users/luisfdiaz@bcp.com.pe/Modulo de Seguimiento/')

import SegScore as rmm
importlib.reload(rmm)

# Librerias y fuciones generales
from pyspark.sql.functions import date_format, expr, to_date, date_sub, add_months, col, when, coalesce, trim, broadcast, avg, max, min, lit, concat, window, round as colround, upper, abs as sparkabs,greatest
from pyspark.sql import functions as F
from pyspark import StorageLevel
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from pyspark.sql.window import Window

from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("Optimized") \
    .config("spark.executor.memory", "32g") \
    .config("spark.executor.cores", "4") \
    .config("spark.executor.instances", "2") \
    .config("spark.sql.shuffle.partitions", "200") \
    .getOrCreate()

In [0]:
#Mostrar los duplicados
def print_res(sparkf_df):
  import pandas as pd
  pd.set_option('display.max_rows', None)
  pd.set_option('display.max_columns', None)
  pd_spark_df = sparkf_df.toPandas()
 
  return pd_spark_df

In [0]:
from pyspark.sql.functions import coalesce, lit

def generar_codmes(inicio, final):
    codmes_list = []
    anio_inicial = inicio // 100
    mes_inicial = inicio % 100
    
    anio_final = final // 100
    mes_final = final % 100
    
    anio = anio_inicial
    mes = mes_inicial
    
    # Generar la lista hasta el mes final
    while (anio < anio_final) or (anio == anio_final and mes <= mes_final):
        codmes_list.append(anio * 100 + mes)
        mes += 1
        if mes > 12:
            mes = 1  # Pasar a enero del siguiente año
            anio += 1
    return codmes_list

def dias_atraso(base, identificador, inicio, final): 
    df_list = []
    espacio = generar_codmes(inicio = inicio, final = final)
    for codmes in espacio:
        numeromes = (codmes // 100) * 12 + (codmes % 100)
        codmes_f1 = ((numeromes - 1 + 1) // 12) * 100 + ((numeromes + 11 - 11 * 1) % 12) + 1
        codmes_f2 = ((numeromes - 1 + 2) // 12) * 100 + ((numeromes + 11 - 11 * 2) % 12) + 1
        codmes_f3 = ((numeromes - 1 + 3) // 12) * 100 + ((numeromes + 11 - 11 * 3) % 12) + 1

        df_base = base.filter(f"CODMES = {codmes}").select("CODMES", identificador, "CTDMESMADURACION", "MTOSALDOCAPITALSOL","TIPBLOQUEOPRODUCTO", "CTDDIAATRASO")
        df_f1 = base.filter(f"CODMES = {codmes_f1}").select("CODMES", identificador, "CTDDIAATRASO")
        df_f2 = base.filter(f"CODMES = {codmes_f2}").select("CODMES", identificador, "CTDDIAATRASO")
        df_f3 = base.filter(f"CODMES = {codmes_f3}").select("CODMES", identificador, "CTDDIAATRASO")

        df_parcial = df_base.alias('A') \
                .join(df_f1.alias('B'), on = identificador, how = 'left') \
                .join(df_f2.alias('C'), on = identificador, how = 'left') \
                .join(df_f3.alias('D'), on = identificador, how = 'left') \
                .selectExpr('A.*', 
                            'COALESCE(B.CTDDIAATRASO, 0) AS CTDDIAATRASO_F1', 
                            'COALESCE(C.CTDDIAATRASO, 0) AS CTDDIAATRASO_F2', 
                            'COALESCE(D.CTDDIAATRASO, 0) AS CTDDIAATRASO_F3')
                
        df_list.append(df_parcial)
        print(f'Iteración {codmes}:', f't0:{codmes}', f't1:{codmes_f1}', f't2:{codmes_f2}', f't3:{codmes_f3}')

    df_final = df_list[0]
    for df in df_list[1:]:
        df_final = df_final.unionByName(df)

    return df_final

In [0]:
def rezago_codmes_col(base, rezago_mes):
    for i in range(0, rezago_mes + 1, 1):
        base = base.withColumn(f"CODMES_{i}", date_format(add_months(to_date(concat(col("codmes"), lit("01")), "yyyyMMdd"), -i), "yyyyMM"))
        base = base.withColumn(f"CODMES_{i}", col(f"CODMES_{i}").cast("integer"))
    return base

def avance_codmes_col(base, avance_mes):
    for l in range(0, avance_mes + 1, 1):
        base = base.withColumn(f"CODMES_F{l}", date_format(add_months(to_date(concat(col("codmes"), lit("01")), "yyyyMMdd"), l), "yyyyMM"))
        base = base.withColumn(f"CODMES_F{l}", col(f"CODMES_F{l}").cast("integer"))
    return base

# Base de seguimiento usual

In [0]:
base_hip = spark.table('catalog_lhcl_prod_bcp.bcp_edv_fabseg.t39290_base_app_hip')

# Filtro inicial
base_hip = base_hip.filter(~base_hip.TIPCREDITO.isin('HVIC', 'HVIM', 'HBIC', 'HSIC', 'HMIC', 'HMIM', 'HREF'))
base_hip = base_hip.drop('MIN_MES_DEF_120_BLOQ_REF_ACT')

# Eliminamos registros especificos
base_hip = base_hip.filter(
    ~(
        (base_hip.CODMES == 202412) &
        (base_hip.REGION == 'MISS') &
        (base_hip.CODCLAVEOPECTA.isin(
            329918345, 330318674, 329976162, 330387465, 329138480, 329138511,
            330318622, 329976170, 329918227, 329918736, 330318689, 330151937,
            329666794, 330318703, 329976146
        ))
    )
)

# Verficamos duplicados
base_hip = base_hip.dropDuplicates(subset=["codmes", "CODCLAVEOPECTA"])
base_hip.persist(StorageLevel.MEMORY_AND_DISK).count()
dup1 =base_hip.persist(StorageLevel.MEMORY_AND_DISK).groupBy("codmes", "CODCLAVEOPECTA").count().alias("count").filter(col("count")>1)
print_res(dup1)

## Llaves cloud

In [0]:
# Pegamos llaves de databricks
md_relclientedl = spark.table('catalog_lhcl_prod_bcp.bcp_edv_fabseg.t39290_md_relclientedl')
md_relcodclaveopectadl = spark.table('catalog_lhcl_prod_bcp.bcp_edv_fabseg.t39290_md_relcodclaveopectadl')

# Pegando codclaveunicocli
base_hip = base_hip.join(
    md_relclientedl.selectExpr("CODCLAVECIC", "CODCLAVEUNICOCLIDL AS CODCLAVEUNICOCLI"),
    on = 'CODCLAVECIC',
    how = 'left'
)

base_hip = base_hip.join(
    md_relcodclaveopectadl.selectExpr("CODCLAVEOPECTA", "CODCLAVECTADL AS CODCLAVECTA"),
    on = 'CODCLAVEOPECTA',
    how = 'left'
)

base_hip.persist(StorageLevel.MEMORY_AND_DISK).count()
dup2 =base_hip.persist(StorageLevel.MEMORY_AND_DISK).groupBy("codmes", "CODCLAVEOPECTA").count().alias("count").filter(col("count")>1)
print_res(dup2)

## Actualización de default

In [0]:
# Tabla mm_mes_primer_default_norev: Obtener el minimo mes de default
mm_mes_primer_default_norev = spark.table('catalog_lhcl_prod_bcp.bcp_ddv_adrmmgr_seginfobasesgenerales_vu.mm_portafoliocredito')

from pyspark.sql.functions import col, coalesce, lit

base_hip = base_hip.join(
    mm_mes_primer_default_norev.select(
        "CODCLAVECTA", 
        coalesce(col("CODMESMINDEF60BLOQUEOREFIN"), lit(999999)).alias("MIN_MES_DEF_60_BLOQ_REF_ACT"), 
        coalesce(col("CODMESMINDEF120BLOQUEOREFIN"), lit(999999)).alias("MIN_MES_DEF_120_BLOQ_REF_ACT")
    ),
    on="CODCLAVECTA",
    how='left'
)

base_hip.persist(StorageLevel.MEMORY_AND_DISK).count()

# Verificando duplicandos en el segundo Join
dup = base_hip.groupBy("codmes", "codclaveopecta").count().alias("count").filter(col("count")>1)
print_res(dup)

In [0]:
# DISTANCIA 'MIN_MES_DEF_120_BLOQ_REF_ACT' 120_24
base_hip = base_hip.withColumn(
    "DISTANCIA_DEF24",
    (F.floor(F.col('MIN_MES_DEF_120_BLOQ_REF_ACT') / 100) * 12 + (F.col('MIN_MES_DEF_120_BLOQ_REF_ACT') - F.floor(F.col('MIN_MES_DEF_120_BLOQ_REF_ACT') / 100) * 100) - 
     (F.floor(F.col("CODMES") / 100) * 12 + (F.col("CODMES") - F.floor(F.col("CODMES") / 100) * 100)))
)

# Crear columnas DEF24 a DEF6
for i in range(24, 5, -1):
    base_hip = base_hip.withColumn(f"DEF{i}", F.when(F.col("DISTANCIA_DEF24") <= i, 1).otherwise(0))

# 'MIN_MES_DEF_120_BLOQ_REF_ACT' TEMPRANO 60_12
base_hip = base_hip.withColumn(
    "DISTANCIA_DEF60_12",
    (F.floor(F.col('MIN_MES_DEF_120_BLOQ_REF_ACT') / 100) * 12 + (F.col('MIN_MES_DEF_120_BLOQ_REF_ACT') - F.floor(F.col('MIN_MES_DEF_120_BLOQ_REF_ACT') / 100) * 100) - 
     (F.floor(F.col("CODMES") / 100) * 12 + (F.col("CODMES") - F.floor(F.col("CODMES") / 100) * 100)))
)
base_hip = base_hip.withColumn("DEF60_12", F.when(F.col("DISTANCIA_DEF60_12") <= 12, 1).otherwise(0))

## Actualización de atrasos

In [0]:
# # Tabla de cuentas ALS: Obtener el atraso en meses posteriores al observado
# hm_univ_ctas_als = spark.table('catalog_lhcl_prod_bcp.bcp_ddv_adrmmgr_seginfobasesgenerales_vu.hm_portafoliocredito')
# hm_univ_ctas_als = hm_univ_ctas_als.filter("codapp = 'ALS'")

# # Backlog Lesly: Automatizar codmes
# # función dias_atraso difinida al inicio
# v_hm_univ_ctas_als = dias_atraso(base=hm_univ_ctas_als, identificador='codclavecta', inicio=201701, final=202504)

# # Cruzando para homologar el opecta
# v_hm_univ_ctas_als = v_hm_univ_ctas_als.join(
#     md_relcodclaveopectadl.selectExpr("CODCLAVECTADL AS CODCLAVECTA", "CODCLAVEOPECTA"),
#     on = "CODCLAVECTA",
#     how = 'left'
# )

# v_hm_univ_ctas_als.createOrReplaceTempView('ctas_als')

# spark.sql('''
#     CREATE OR REPLACE TABLE
#         catalog_lhcl_prod_bcp.bcp_edv_fabseg.TABLASGE_v_hm_univ_ctas_als
#     USING DELTA LOCATION 
#         'abfss://bcp-edv-fabseg@adlscu1lhclbackp05.dfs.core.windows.net/TABLASGE_SEG_PER/v_hm_univ_ctas_als'
#     AS SELECT * FROM ctas_als
#     '''
# )

In [0]:
v_hm_univ_ctas_als = spark.table('catalog_lhcl_prod_bcp.bcp_edv_fabseg.TABLASGE_v_hm_univ_ctas_als')
base_hip = base_hip.join(
    v_hm_univ_ctas_als, 
    (base_hip.CODMES == v_hm_univ_ctas_als.CODMES) & 
    (base_hip['CODCLAVECTA'] == v_hm_univ_ctas_als['CODCLAVECTA']), 
    'left'
).select(
    base_hip['*'], 
    v_hm_univ_ctas_als.CTDDIAATRASO_F1, 
    v_hm_univ_ctas_als.CTDDIAATRASO_F2, 
    v_hm_univ_ctas_als.CTDDIAATRASO_F3
)

base_hip.persist(StorageLevel.MEMORY_AND_DISK).count()

# Verificando duplicandos en el segundo Join
dup = base_hip.groupBy("codmes", "CODCLAVECTA").count().alias("count").filter(col("count")>1)
print_res(dup)

In [0]:
base_hip = base_hip.withColumn("MORA_8_3", F.when(F.greatest("CTDDIAATRASO_F1", "CTDDIAATRASO_F2", "CTDDIAATRASO_F3") >= 8, 1).otherwise(0))
base_hip = base_hip.withColumn("MORA_30_3", F.when(F.greatest("CTDDIAATRASO_F1", "CTDDIAATRASO_F2", "CTDDIAATRASO_F3") >= 30, 1).otherwise(0))
base_hip = base_hip.withColumn("MORA_45_3", F.when(F.greatest("CTDDIAATRASO_F1", "CTDDIAATRASO_F2", "CTDDIAATRASO_F3") >= 45, 1).otherwise(0))
base_hip = base_hip.withColumn("MORA_60_3", F.when(F.greatest("CTDDIAATRASO_F1", "CTDDIAATRASO_F2", "CTDDIAATRASO_F3") >= 60, 1).otherwise(0))

## Marca de tipo de fondeo y corrigiendo segmento

In [0]:
# FLAG DE TIPO DE FONDEO
base_hip_1 = base_hip.withColumn(
    "TIPO_FONDEO",
    F.when(F.col("TIPCREDITO").isin(['TH01','TH02','HICC','RHIP','RREF','HICB','HIFS','HIBS','HIBK','HIPA','HIPD', 'HIPO', 'HIPP', 'HIPT','HIPR', 'HCPO', 'HITR','HTIV']), "FONDEO BCP")
     .when(F.col("TIPCREDITO").isin(['GAHI', 'GAHP', 'GAHR', 'GAPC']), "GAHI")
     .when(F.col("TIPCREDITO").isin(['HITP', 'HMIV', 'HVIV', 'HSIV', 'HBIV']), "FONDEO MVIV")
     .otherwise("OTROS")
)

base_hip_1 = base_hip_1.withColumn(
    "SEGMENTOAGRUPADO_CORREGIDO_C",
    F.when(F.trim(F.when(F.col("SEGMENTOCORREGIDO").isNull(), F.col("CODCAMPANIA")).otherwise(F.col("SEGMENTOCORREGIDO"))).isin(['E15','E45','F41','F94','A62','F70','H27','J68','K28']), 'AHORRO LOCAL')
     .when(F.trim(F.when(F.col("SEGMENTOCORREGIDO").isNull(), F.col("CODCAMPANIA")).otherwise(F.col("SEGMENTOCORREGIDO"))).isin(['E46','F14','H26']), 'GREMIO BCP')
     .when(F.trim(F.when(F.col("SEGMENTOCORREGIDO").isNull(), F.col("CODCAMPANIA")).otherwise(F.col("SEGMENTOCORREGIDO"))).isin(['E44','F26','H25','H42','H35','K24','K27']), 'PYME')
     .when(F.trim(F.when(F.col("SEGMENTOCORREGIDO").isNull(), F.col("CODCAMPANIA")).otherwise(F.col("SEGMENTOCORREGIDO"))).isin(['F36','F42']), 'REMESAS')
     .when(F.trim(F.when(F.col("SEGMENTOCORREGIDO").isNull(), F.col("CODCAMPANIA")).otherwise(F.col("SEGMENTOCORREGIDO"))).isin(['860','E06','E11','E42','E43','E47','F32','F33','F34','F35','F37','F38','F39','F40','F43','F44','F45','F48','F84','G04','G05','208','70','C12','F83','E35','I15','J91','J92','E56','K23','K26']), 'TRADICIONAL')
     .otherwise('OTROS')
)

## Tabla de estados

In [0]:
sol_estados_ant = spark.sql("""
SELECT 
    DISTINCT t.CODSOLICITUD, t.SCORE_APPLICANT_FINAL, t.INGRESO_TITULAR, t.INGRESO_CONYUGE, t.DEPARTAMENTO, t.PROVINCIA, t.DISTRITO, t.CODEMP, t.CODCLAVECIC_E
FROM 
    (SELECT 
        a.*, 
        CASE 
            WHEN fecactualizaciontabla = '06FEB2023:16:58:26' THEN 2 
            WHEN fecactualizaciontabla = '03JAN2024:00:00:00' THEN 1 
            WHEN fecactualizaciontabla IS NULL THEN 0 
        END AS fecha_act 
     FROM 
        catalog_lhcl_prod_bcp.bcp_edv_fabseg.T45988_HM_SOLICITUDES_CH_RMS_ESTADOS_202504 AS a
     WHERE 
        SCORE_APPLICANT_FINAL IS NOT NULL) t
INNER JOIN 
    (SELECT 
        codsolicitud, 
        MAX(CASE 
            WHEN fecactualizaciontabla = '06FEB2023:16:58:26' THEN 2 
            WHEN fecactualizaciontabla = '03JAN2024:00:00:00' THEN 1 
            WHEN fecactualizaciontabla IS NULL THEN 0 
        END) AS max_fechaactualizacion
     FROM 
        catalog_lhcl_prod_bcp.bcp_edv_fabseg.T45988_HM_SOLICITUDES_CH_RMS_ESTADOS_202504
     WHERE 
        SCORE_APPLICANT_FINAL IS NOT NULL
     GROUP BY 
        codsolicitud) sub
ON 
    t.codsolicitud = sub.codsolicitud AND t.fecha_act = sub.max_fechaactualizacion
""")
sol_estados_ant.persist(StorageLevel.MEMORY_AND_DISK).count()

In [0]:
tabla_suite_unique = sol_estados_ant.dropDuplicates(['CODSOLICITUD']).select(
    'CODSOLICITUD',
    'SCORE_APPLICANT_FINAL',
    F.col('INGRESO_TITULAR').alias('ESTADOS_INGRESO_TITULAR'),
    F.col('INGRESO_CONYUGE').alias('ESTADOS_INGRESO_CONYUGE'),
    F.col('DEPARTAMENTO').alias('ESTADOS_DEPARTAMENTO'),
    F.col('PROVINCIA').alias('ESTADOS_PROVINCIA'),
    F.col('DISTRITO').alias('ESTADOS_DISTRITO'),
    F.col('CODEMP').alias('ESTADOS_CODEMP'),
    F.col('CODCLAVECIC_E').alias('ESTADOS_CODCLAVECIC_E')
) 

tabla_suite_unique.persist(StorageLevel.MEMORY_AND_DISK).count()

# Verificando duplicandos en el segundo Join
dup = tabla_suite_unique.groupBy("CODSOLICITUD").count().alias("count").filter(col("count")>1)
print_res(dup)

In [0]:
base_hip_2 = base_hip_1.join(
    tabla_suite_unique,
    on = ["CODSOLICITUD"],
    how = 'left'
)

base_hip_2 = base_hip_2.withColumn("SC_ESTADOS", F.col("SCORE_APPLICANT_FINAL")) \
                       .withColumn("XB_ESTADOS", (174.25 - F.col("SCORE_APPLICANT_FINAL")) / 57.708) \
                       .withColumn("PD_ESTADOS", 1 / (1 + F.exp(-F.col("XB_ESTADOS")))) 

base_hip_2.persist(StorageLevel.MEMORY_AND_DISK).count()

# Verificando duplicandos en el segundo Join
dup = base_hip_2.groupBy("CODMES", "CODCLAVEOPECTA").count().alias("count").filter(col("count")>1)
print_res(dup)

## Duplicados de codsolicitud

In [0]:
dup = base_hip_2.filter(F.col("CODSOLICITUD").isNotNull()) \
  .groupBy("CODMES", "CODSOLICITUD") \
  .agg(F.count("*").alias("N")) \
  .filter(F.col("N") > 1)

# Crear la tabla SOL
sol = dup.select("CODSOLICITUD").distinct()

# Crear la tabla R.DUPLICADOS_SOLICITUD
duplicados_solicitud = base_hip_2.alias("A") \
    .join(sol.alias("B"), F.col("A.CODSOLICITUD") == F.col("B.CODSOLICITUD"), "inner") \
    .select("A.*")

# Importando tabla MD_prestamo
md_prestamo = spark.table('catalog_lhcl_prod_bcp.bcp_udv_int_vu.m_cuentacreditopersonal')

md_prestamo = md_prestamo.select(
    col("CODCLAVECTA"),
    date_format(col("FECCIERRE"), "yyyyMM").cast("int").alias("CODMESCIERRE"),
    date_format(col("FECCANCELACION"), "yyyyMM").cast("int").alias("CODMESCANCEL")
)

In [0]:
# Cruce para explorar fechas de cierre y fecha de cancelación
duplicados_solicitud_1 = broadcast(duplicados_solicitud).join(
    md_prestamo,
    on = "CODCLAVECTA",
    how = "left"
).select(
    "*",
    coalesce(col("CODMESCIERRE"), lit(999999)).alias("CODMESCIERRE_F"),
    coalesce(col("CODMESCANCEL"), lit(999999)).alias("CODMESCANCEL_F")
)

# Agregar las columnas FLG_CIERRE_CTA, FLG_CANCELADA_CTA y FLG_ELIMINAR
duplicados_solicitud_2 = duplicados_solicitud_1.withColumn("FLG_CIERRE_CTA", F.when(F.col("CODMESCIERRE_F") == 999999, 0).otherwise(1)) \
    .withColumn("FLG_CANCELADA_CTA", F.when(F.col("CODMESCANCEL_F") == 999999, 0).otherwise(1)) \
    .withColumn("FLG_ELIMINAR", F.when(F.col("CODMESCANCEL_F") == F.col("CODMES"), 1).otherwise(0))

# Crear la tabla DUPLICADOS_SOLICITUD_3
duplicados_solicitud_3 = duplicados_solicitud_2.filter(F.col("FLG_ELIMINAR") == 1)

In [0]:
# Realizar el join y filtrar las filas donde CODCLAVEOPECTA es nulo en duplicados_solicitud_3
base_hip_3 = base_hip_2.alias("A") \
    .join(duplicados_solicitud_3.alias("B"), F.col("A.CODCLAVEOPECTA") == F.col("B.CODCLAVEOPECTA"), "left") \
    .filter(F.col("B.CODCLAVEOPECTA").isNull()) \
    .select("A.*")

base_hip_3.persist(StorageLevel.MEMORY_AND_DISK).count()

# Verificando duplicandos en el segundo Join
dup = base_hip_3.groupBy("codmes", "codclaveopecta").count().alias("count").filter(col("count")>1)
print_res(dup)

## Gremio

In [0]:
!pip install oracledb
import oracledb
import pandas as pd

# Datos de conexión
username = "T39290"
password = "Vjqgqg$9171#"
dsn = "130.1.22.91:1521/BCPDW3"

# Configuramos la conexión a oracle
connection = oracledb.connect(user=username, password=password, dsn=dsn)

# Query de extracción
query = "SELECT * FROM ODS_V.MD_EMPLEADO"

chunks = pd.read_sql(query, connection, chunksize=3000)

# Unir todos los chunks en un solo DataFrame
md_empleado = pd.concat(chunks)

# Cerrar la conexión
connection.close()

# Resumiendo gremio
gremio = md_empleado.groupby('CODCLAVECIC').agg({ 'FECINGRESOEMPLEADO': 'min', 'FECRETIROEMPLEADO': 'max' }).reset_index()
gremio['FECINGRESOEMPLEADO_F'] = gremio['FECINGRESOEMPLEADO'].dt.year * 100 + gremio['FECINGRESOEMPLEADO'].dt.month
gremio['FECRETIROEMPLEADO_F'] = gremio['FECRETIROEMPLEADO'].dt.year * 100 + gremio['FECRETIROEMPLEADO'].dt.month

gremio = spark.createDataFrame(gremio)

In [0]:
base_hip_4 = base_hip_3.alias("A") \
    .join(gremio.alias("G"), F.col("A.CODCLAVECIC") == F.col("G.CODCLAVECIC"), "left") \
    .select(
        "A.*",
        F.when(F.col("G.CODCLAVECIC").isNull(), 0)
         .when((F.col("A.CODMES") >= F.col("G.FECINGRESOEMPLEADO_F")) & 
               ((F.col("A.CODMES") <= F.col("G.FECRETIROEMPLEADO_F")) | F.col("G.FECRETIROEMPLEADO").isNull()), 1)
         .otherwise(0).alias("FLG_GREMIO_F")
    )

base_hip_4.persist(StorageLevel.MEMORY_AND_DISK).count()

# Verificando duplicandos en el segundo Join
dup = base_hip_4.groupBy("codmes", "codclaveopecta").count().alias("count").filter(col("count")>1)
print_res(dup)

## Drivers base

In [0]:
# Default foto
base_hip_4 = base_hip_4.withColumn(
    "DEF_120_BLOQ_REF_0",
    F.when(F.col("CODMES") < F.col("MIN_MES_DEF_120_BLOQ_REF_ACT"), 0).otherwise(1)
)

# Marca HML
base_hip_4 = base_hip_4.withColumn(
    "MARCA_HML_T",
    F.when(F.col("MARCA_HML").isNull(), '99. Missing')
     .when(F.col("MARCA_HML").isin('H+', 'H-'), '1. H')
     .when(F.col("MARCA_HML").isin('M+', 'M-'), '2. M')
     .when(F.col("MARCA_HML").isin('L+', 'L-'), '3. L')
     .otherwise('98. Otro')
)

# Marca segmento banca
base_hip_4 = base_hip_4.withColumn(
    "SEGMENTO_BANCA",
    F.when(F.col("CODSUBSEGMENTO").isNull(), '99. Missing')
     .when(F.col("CODSUBSEGMENTO") == 'X1N', '1. Bex')
     .when(F.col("CODSUBSEGMENTO").isin('P1N', 'LEN'), '2. Enalta/Privada')
     .when(F.col("CODSUBSEGMENTO") == 'M1N', '3. Consumo')
     .when(F.col("CODSUBSEGMENTO") == 'Q1N', '4. Pyme')
     .when(F.col("CODSUBSEGMENTO") == 'NBN', '5. No Banco')
     .otherwise('98. Otros')
)

base_hip_4 = base_hip_4.withColumn(
    "SEGMENTO_BANCA_F",
    F.when(F.col("CODSUBSEGMENTO").isNull(), '99. Missing')
     .when(F.col("CODSUBSEGMENTO").isin('X1N', 'P1N', 'LEN'), '1. Afluente')
     .when(F.col("CODSUBSEGMENTO").isin('M1N', 'Q1N'), '2. Consumo')
     .otherwise('3. Resto')
)

# Rango de edad
base_hip_4 = base_hip_4.withColumn(
    "EDAD_R",
    F.when(F.col("EDAD").isNull(), '99. Missing')
     .when(F.col("EDAD") <= 30, '1. [00 - 30]')
     .when(F.col("EDAD") <= 35, '2. <30 - 35]')
     .when(F.col("EDAD") <= 40, '3. <35 - 40]')
     .when(F.col("EDAD") <= 48, '4. <40 - 48]')
     .otherwise('5. <48 - ++]')
)

# Estado civil
base_hip_4 = base_hip_4.withColumn(
    "EST_CIVIL_R",
    F.when(F.col("TIPESTCIVIL").isNull(), '99. Missing')
     .when(F.col("TIPESTCIVIL") == 'SOL', '1. Soltero')
     .when(F.col("TIPESTCIVIL") == 'CAS', '2. Casado')
     .otherwise('98. Otros')
)

# Rango plazo
base_hip_4 = base_hip_4.withColumn(
    "PLAZO_R",
    F.when(F.col("CTDPLAZOAPROBADO").isNull(), '99. Missing')
     .when(F.col("CTDPLAZOAPROBADO") <= 10, '1. [0 - 10]')
     .when(F.col("CTDPLAZOAPROBADO") <= 15, '2. <10 - 15]')
     .when(F.col("CTDPLAZOAPROBADO") <= 20, '3. <15 - 20]')
     .when(F.col("CTDPLAZOAPROBADO") <= 25, '4. <20 - 25]')
     .otherwise('5. <25 - ++]')
)

# Endeudamiento total en el sistema financiero incluyendo el ingreso del cónyuge
base_hip_4 = base_hip_4.withColumn(
    "END_TOT_SF_CY",
    (F.coalesce(F.col("DEU_RCC_TC_SBS"), F.lit(0)) +
     F.coalesce(F.col("DEU_RCC_CEF_SBS"), F.lit(0)) +
     F.coalesce(F.col("DEU_RCC_VEH_SBS"), F.lit(0)) +
     F.coalesce(F.col("DEU_RCC_HIP_SBS"), F.lit(0))) / F.coalesce(F.col("INGRESO_SOL_CONY"), F.lit(1000))
)

base_hip_4 = base_hip_4.withColumn(
    "END_TOT_SF_CY_R",
    F.when(F.col("END_TOT_SF_CY") == 0, '1. [0]')
     .when(F.col("END_TOT_SF_CY") <= 1, '2. <0 - 1]')
     .when(F.col("END_TOT_SF_CY") <= 4, '3. <1 - 4]')
     .when(F.col("END_TOT_SF_CY") <= 6, '4. <4 - 6]')
     .when(F.col("END_TOT_SF_CY") <= 8, '5. <6 - 8]')
     .when(F.col("END_TOT_SF_CY") <= 12, '6. <8 - 12]')
     .otherwise('7. <12 - ++>')
)

# Endeudamiento en el sistema financiero con solo ingreso del titular
base_hip_4 = base_hip_4.withColumn(
    "END_TOT_SF",
    (F.coalesce(F.col("DEUDA_TC_RBM"), F.lit(0)) +
     F.coalesce(F.col("DEUDA_CEF_RBM"), F.lit(0)) +
     F.coalesce(F.col("DEUDA_VEH_RBM"), F.lit(0))) / F.coalesce(F.col("INGRESO_RBM"), F.lit(1000))
)

base_hip_4 = base_hip_4.withColumn(
    "END_TOT_SF_R",
    F.when(F.col("END_TOT_SF") == 0, '1. [0]')
     .when(F.col("END_TOT_SF") <= 1, '2. <0 - 1]')
     .when(F.col("END_TOT_SF") <= 4, '3. <1 - 4]')
     .when(F.col("END_TOT_SF") <= 6, '4. <4 - 6]')
     .when(F.col("END_TOT_SF") <= 8, '5. <6 - 8]')
     .when(F.col("END_TOT_SF") <= 12, '6. <8 - 12]')
     .otherwise('7. <12 - ++>')
)

# Moneda
base_hip_4 = base_hip_4.withColumn(
    "MONEDA",
    F.when(F.col("CODMONEDA").isNull(), '99.Missing')
     .when(F.col("CODMONEDA") == '0001', '01. Soles')
     .when(F.col("CODMONEDA") == '1001', '02. Dolares')
)

# Rango ingreso titular
base_hip_4 = base_hip_4.withColumn(
    "RAN_ING_TIT",
    F.when(F.col("INGRESO_RBM").isNull(), '99. Missing')
     .when(F.col("INGRESO_RBM") <= 2500, '1. [0 - 2,500]')
     .when(F.col("INGRESO_RBM") <= 5000, '2. <2,500 - 5,000]')
     .when(F.col("INGRESO_RBM") <= 7500, '3. <5,000 - 7,500]')
     .when(F.col("INGRESO_RBM") <= 10000, '4. <7,500 - 10,000]')
     .when(F.col("INGRESO_RBM") <= 20000, '5. <10,000 - 20,000]')
     .otherwise('6. <20,000 - ++>')
)

# Rango ingreso conyugal
base_hip_4 = base_hip_4.withColumn(
    "RAN_ING_CY",
    F.when(F.col("INGRESO_SOL_CONY").isNull(), '99. Missing')
     .when(F.col("INGRESO_SOL_CONY") <= 2500, '1. [0 - 2,500]')
     .when(F.col("INGRESO_SOL_CONY") <= 5000, '2. <2,500 - 5,000]')
     .when(F.col("INGRESO_SOL_CONY") <= 7500, '3. <5,000 - 7,500]')
     .when(F.col("INGRESO_SOL_CONY") <= 10000, '4. <7,500 - 10,000]')
     .when(F.col("INGRESO_SOL_CONY") <= 20000, '5. <10,000 - 20,000]')
     .otherwise('6. <20,000 - ++>')
)

# Complementando el CAMPANIA_AGRUPADA como lo hace IFRS9
base_hip_4 = base_hip_4.withColumn(
    "CAMPANIA_AGRUPADA_NEW",
    F.when(F.col("CAMPANIA_AGRUPADA").isNull() & F.col("TIPCREDITO").isin('HTIV','HVIV', 'HVIC', 'HVIM', 'HBIV', 'HBIC','HSIV', 'HSIC', 'HMIV', 'HMIC', 'HMIM', 'HITP'), 'Ahorro Local')
     .when(F.col("CAMPANIA_AGRUPADA").isNull() & F.col("TIPCREDITO").isin('HCPO', 'HIPA', 'HIPD', 'HIPO', 'HIPP','HIPR', 'HIPT', 'HIBS', 'HIBK', 'HREF', 'RHIP', 'HICC', 'HICB', 'HITR', 'HIFS', 'TH01', 'TH02'), 'Tradicional')
     .when(F.col("TIPCREDITO").isin('GAHI','GAHP','GAPC'), 'GAHI')
     .otherwise(F.col("CAMPANIA_AGRUPADA"))
)

## Incidencias CEF

In [0]:
incidencias_cef = spark.table('catalog_lhcl_prod_bcp.bcp_edv_fabseg.T45988_INCIDENCIAS_CEF_202504')

base_hip_4 = base_hip_4.join(
    incidencias_cef.select("CODMES", "CODCLAVECIC", F.lit(1).alias("FLG_INCIDENCIA")),
    on=["CODMES", "CODCLAVECIC"],
    how='left'
)

base_hip_4 = base_hip_4.withColumn(
    'FLG_INCIDENCIA', 
    F.when(base_hip_4['FLG_INCIDENCIA'].isNull(), 0).otherwise(1)
)

base_hip_4.persist(StorageLevel.MEMORY_AND_DISK).count()

dup = base_hip_4.groupBy("codmes", "codclaveopecta").count().alias("count").filter(col("count")>1)
print_res(dup)

## Tasa y cuota

In [0]:
md_prestamo = spark.table('catalog_lhcl_prod_bcp.bcp_udv_int_vu.m_cuentacreditopersonal')
base_hip_5 = base_hip_4.join(
    md_prestamo.select(
        'codclavecta',
        col('pcttasainteresoriginal').alias('PRS_TASA'),
        col('ctdcuotaanual').alias('PRS_CUOTA_ANUAL'),
        col('ctdmesoriginal').alias('PRS_PLAZO'),
        col('mtovalorbien').alias('PRS_ValorComVivienda')
    ),
    on='codclavecta',
    how='left'
)

base_hip_5.persist(StorageLevel.MEMORY_AND_DISK).count()

dup = base_hip_5.groupBy("codmes", "codclaveopecta").count().alias("count").filter(col("count")>1)
print_res(dup)

## CEM

In [0]:
base_hip_5 = base_hip_5.withColumn("CODCLAVEUNICOCLI", 
                   when(col("CODCLAVECIC") == 22780207, "36b6035f939a9a07ad3d57b052367685590ef97d3f23c6851837cb95f3dc57f10082bba27dd0ba3e05a29d8d7651f72b74b8bda2e862b9098e51fcbb8aa2c2ee")
                   .when(col("CODCLAVECIC") == 9111029, "2f0d282125c78c876d75120067edf198b0c2eacbea149329addb2c659657be0d8e27e9f684f7308c59e7d5cf6ef794673498e7ddf3bc41e73700b034eeebcad6")
                   .otherwise(col("CODCLAVEUNICOCLI")))

In [0]:
base_hip_5.createOrReplaceTempView("base_hip_5")
base_hip_6 = spark.sql(f"""
     select A.*,
     B_0.MTOCEMCLISOL AS MTOCEMCLISOL_0,
     B_1.MTOCEMCLISOL AS MTOCEMCLISOL_1,
     B_2.MTOCEMCLISOL AS MTOCEMCLISOL_2,
     B_3.MTOCEMCLISOL AS MTOCEMCLISOL_3,
     B_4.MTOCEMCLISOL AS MTOCEMCLISOL_4,
     B_5.MTOCEMCLISOL AS MTOCEMCLISOL_5,
     B_6.MTOCEMCLISOL AS MTOCEMCLISOL_6,
     B_F3.MTOCEMCLISOL AS MTOCEMCLISOL_F3,
     B_F6.MTOCEMCLISOL AS MTOCEMCLISOL_F6,
     B_0.MTOCUOTATOTALGARCGTESOL AS MTOCUOTATOTALGARCGTESOL_0,
     B_1.MTOCUOTATOTALGARCGTESOL AS MTOCUOTATOTALGARCGTESOL_1,
     B_2.MTOCUOTATOTALGARCGTESOL AS MTOCUOTATOTALGARCGTESOL_2,
     B_3.MTOCUOTATOTALGARCGTESOL AS MTOCUOTATOTALGARCGTESOL_3,
     B_4.MTOCUOTATOTALGARCGTESOL AS MTOCUOTATOTALGARCGTESOL_4,
     B_5.MTOCUOTATOTALGARCGTESOL AS MTOCUOTATOTALGARCGTESOL_5,
     B_6.MTOCUOTATOTALGARCGTESOL AS MTOCUOTATOTALGARCGTESOL_6,
     B_F3.MTOCUOTATOTALGARCGTESOL AS MTOCUOTATOTALGARCGTESOL_F3,
     B_F6.MTOCUOTATOTALGARCGTESOL AS MTOCUOTATOTALGARCGTESOL_F6,
     B_0.MTOAJUSTADOBCISOL AS MTOAJUSTADOBCISOL_0,
     B_1.MTOAJUSTADOBCISOL AS MTOAJUSTADOBCISOL_1,
     B_2.MTOAJUSTADOBCISOL AS MTOAJUSTADOBCISOL_2,
     B_3.MTOAJUSTADOBCISOL AS MTOAJUSTADOBCISOL_3,
     B_4.MTOAJUSTADOBCISOL AS MTOAJUSTADOBCISOL_4,
     B_5.MTOAJUSTADOBCISOL AS MTOAJUSTADOBCISOL_5,
     B_6.MTOAJUSTADOBCISOL AS MTOAJUSTADOBCISOL_6,
     B_0.MTOCUOTACEFBCPSOL AS MTOCUOTACEFBCPSOL_0,
     B_1.MTOCUOTACEFBCPSOL AS MTOCUOTACEFBCPSOL_1,
     B_2.MTOCUOTACEFBCPSOL AS MTOCUOTACEFBCPSOL_2,
     B_3.MTOCUOTACEFBCPSOL AS MTOCUOTACEFBCPSOL_3,
     B_4.MTOCUOTACEFBCPSOL AS MTOCUOTACEFBCPSOL_4,
     B_5.MTOCUOTACEFBCPSOL AS MTOCUOTACEFBCPSOL_5,
     B_6.MTOCUOTACEFBCPSOL AS MTOCUOTACEFBCPSOL_6,
     B_F3.MTOCUOTACEFBCPSOL AS MTOCUOTACEFBCPSOL_F3,
     B_F6.MTOCUOTACEFBCPSOL AS MTOCUOTACEFBCPSOL_F6,
     B_0.MTOCUOTACEFBCPSOL AS MTOCUOTATCBCPSOL_0,
     B_1.MTOCUOTACEFBCPSOL AS MTOCUOTATCBCPSOL_1,
     B_2.MTOCUOTACEFBCPSOL AS MTOCUOTATCBCPSOL_2,
     B_3.MTOCUOTACEFBCPSOL AS MTOCUOTATCBCPSOL_3,
     B_4.MTOCUOTACEFBCPSOL AS MTOCUOTATCBCPSOL_4,
     B_5.MTOCUOTACEFBCPSOL AS MTOCUOTATCBCPSOL_5,
     B_6.MTOCUOTACEFBCPSOL AS MTOCUOTATCBCPSOL_6,
     B_0.MTOCUOTATOTALBCPSOL AS MTOCUOTATOTALBCPSOL_0,
     B_1.MTOCUOTATOTALBCPSOL AS MTOCUOTATOTALBCPSOL_1,
     B_2.MTOCUOTATOTALBCPSOL AS MTOCUOTATOTALBCPSOL_2,
     B_3.MTOCUOTATOTALBCPSOL AS MTOCUOTATOTALBCPSOL_3,
     B_4.MTOCUOTATOTALBCPSOL AS MTOCUOTATOTALBCPSOL_4,
     B_5.MTOCUOTATOTALBCPSOL AS MTOCUOTATOTALBCPSOL_5,
     B_6.MTOCUOTATOTALBCPSOL AS MTOCUOTATOTALBCPSOL_6,
     B_0.MTOCUOTAESTIMADOCEFSBSSINBCPSOL AS MTOCUOTAESTIMADOCEFSBSSINBCPSOL_0,
     B_1.MTOCUOTAESTIMADOCEFSBSSINBCPSOL AS MTOCUOTAESTIMADOCEFSBSSINBCPSOL_1,
     B_2.MTOCUOTAESTIMADOCEFSBSSINBCPSOL AS MTOCUOTAESTIMADOCEFSBSSINBCPSOL_2,
     B_3.MTOCUOTAESTIMADOCEFSBSSINBCPSOL AS MTOCUOTAESTIMADOCEFSBSSINBCPSOL_3,
     B_4.MTOCUOTAESTIMADOCEFSBSSINBCPSOL AS MTOCUOTAESTIMADOCEFSBSSINBCPSOL_4,
     B_5.MTOCUOTAESTIMADOCEFSBSSINBCPSOL AS MTOCUOTAESTIMADOCEFSBSSINBCPSOL_5,
     B_6.MTOCUOTAESTIMADOCEFSBSSINBCPSOL AS MTOCUOTAESTIMADOCEFSBSSINBCPSOL_6,
     B_0.MTOCUOTAESTIMADOTCSBSSINBCPSOL AS MTOCUOTAESTIMADOTCSBSSINBCPSOL_0,
     B_1.MTOCUOTAESTIMADOTCSBSSINBCPSOL AS MTOCUOTAESTIMADOTCSBSSINBCPSOL_1,
     B_2.MTOCUOTAESTIMADOTCSBSSINBCPSOL AS MTOCUOTAESTIMADOTCSBSSINBCPSOL_2,
     B_3.MTOCUOTAESTIMADOTCSBSSINBCPSOL AS MTOCUOTAESTIMADOTCSBSSINBCPSOL_3,
     B_4.MTOCUOTAESTIMADOTCSBSSINBCPSOL AS MTOCUOTAESTIMADOTCSBSSINBCPSOL_4,
     B_5.MTOCUOTAESTIMADOTCSBSSINBCPSOL AS MTOCUOTAESTIMADOTCSBSSINBCPSOL_5,
     B_6.MTOCUOTAESTIMADOTCSBSSINBCPSOL AS MTOCUOTAESTIMADOTCSBSSINBCPSOL_6,
     B_0.MTOCUOTAVEHICULARBCPSOL AS MTOCUOTAVEHICULARBCPSOL_0,
     B_1.MTOCUOTAVEHICULARBCPSOL AS MTOCUOTAVEHICULARBCPSOL_1,
     B_2.MTOCUOTAVEHICULARBCPSOL AS MTOCUOTAVEHICULARBCPSOL_2,
     B_3.MTOCUOTAVEHICULARBCPSOL AS MTOCUOTAVEHICULARBCPSOL_3,
     B_4.MTOCUOTAVEHICULARBCPSOL AS MTOCUOTAVEHICULARBCPSOL_4,
     B_5.MTOCUOTAVEHICULARBCPSOL AS MTOCUOTAVEHICULARBCPSOL_5,
     B_6.MTOCUOTAVEHICULARBCPSOL AS MTOCUOTAVEHICULARBCPSOL_6,
     B_0.MTOCUOTAHIPOTECARIOBCPSOL AS MTOCUOTAHIPOTECARIOBCPSOL_0,
     B_1.MTOCUOTAHIPOTECARIOBCPSOL AS MTOCUOTAHIPOTECARIOBCPSOL_1,
     B_2.MTOCUOTAHIPOTECARIOBCPSOL AS MTOCUOTAHIPOTECARIOBCPSOL_2,
     B_3.MTOCUOTAHIPOTECARIOBCPSOL AS MTOCUOTAHIPOTECARIOBCPSOL_3,
     B_4.MTOCUOTAHIPOTECARIOBCPSOL AS MTOCUOTAHIPOTECARIOBCPSOL_4,
     B_5.MTOCUOTAHIPOTECARIOBCPSOL AS MTOCUOTAHIPOTECARIOBCPSOL_5,
     B_6.MTOCUOTAHIPOTECARIOBCPSOL AS MTOCUOTAHIPOTECARIOBCPSOL_6,
     B_0.MTOCUOTAVIVIENDABCPSOL AS MTOCUOTAVIVIENDABCPSOL_0,
     B_1.MTOCUOTAVIVIENDABCPSOL AS MTOCUOTAVIVIENDABCPSOL_1,
     B_2.MTOCUOTAVIVIENDABCPSOL AS MTOCUOTAVIVIENDABCPSOL_2,
     B_3.MTOCUOTAVIVIENDABCPSOL AS MTOCUOTAVIVIENDABCPSOL_3,
     B_4.MTOCUOTAVIVIENDABCPSOL AS MTOCUOTAVIVIENDABCPSOL_4,
     B_5.MTOCUOTAVIVIENDABCPSOL AS MTOCUOTAVIVIENDABCPSOL_5,
     B_6.MTOCUOTAVIVIENDABCPSOL AS MTOCUOTAVIVIENDABCPSOL_6,
     B_0.MTOCUOTALTOTALSBSSINBCPSOL AS MTOCUOTALTOTALSBSSINBCPSOL_0,
     B_1.MTOCUOTALTOTALSBSSINBCPSOL AS MTOCUOTALTOTALSBSSINBCPSOL_1,
     B_2.MTOCUOTALTOTALSBSSINBCPSOL AS MTOCUOTALTOTALSBSSINBCPSOL_2,
     B_3.MTOCUOTALTOTALSBSSINBCPSOL AS MTOCUOTALTOTALSBSSINBCPSOL_3,
     B_4.MTOCUOTALTOTALSBSSINBCPSOL AS MTOCUOTALTOTALSBSSINBCPSOL_4,
     B_5.MTOCUOTALTOTALSBSSINBCPSOL AS MTOCUOTALTOTALSBSSINBCPSOL_5,
     B_6.MTOCUOTALTOTALSBSSINBCPSOL AS MTOCUOTALTOTALSBSSINBCPSOL_6
     from base_hip_5 as a
     left join catalog_lhcl_prod_bcp.bcp_ddv_rbmrbmper_fuenteinfocli_vu.hm_clientecemrbm as b_0
         on trim(a.CODCLAVEUNICOCLI)=trim(b_0.CODCLAVEUNICOCLI) and a.CODMES=b_0.CODMES
     left join catalog_lhcl_prod_bcp.bcp_ddv_rbmrbmper_fuenteinfocli_vu.hm_clientecemrbm as b_1
         on trim(a.CODCLAVEUNICOCLI)=trim(b_1.CODCLAVEUNICOCLI) and
         DATE_ADD(MONTH, -1, CAST(CONCAT(SUBSTRING(a.CODMES, 1, 4), '-', SUBSTRING(a.CODMES, 5, 2), '-01') AS DATE)) =CONCAT(SUBSTRING(b_1.CODMES, 1, 4), '-', SUBSTRING(b_1.CODMES, 5, 2), '-01')
     left join catalog_lhcl_prod_bcp.bcp_ddv_rbmrbmper_fuenteinfocli_vu.hm_clientecemrbm as b_2
         on trim(a.CODCLAVEUNICOCLI)=trim(b_2.CODCLAVEUNICOCLI) and
         DATE_ADD(MONTH, -2, CAST(CONCAT(SUBSTRING(a.CODMES, 1, 4), '-', SUBSTRING(a.CODMES, 5, 2), '-01') AS DATE)) =CONCAT(SUBSTRING(b_2.CODMES, 1, 4), '-', SUBSTRING(b_2.CODMES, 5, 2), '-01')
     left join catalog_lhcl_prod_bcp.bcp_ddv_rbmrbmper_fuenteinfocli_vu.hm_clientecemrbm as b_3
         on trim(a.CODCLAVEUNICOCLI)=trim(b_3.CODCLAVEUNICOCLI) and
         DATE_ADD(MONTH, -3, CAST(CONCAT(SUBSTRING(a.CODMES, 1, 4), '-', SUBSTRING(a.CODMES, 5, 2), '-01') AS DATE)) =CONCAT(SUBSTRING(b_3.CODMES, 1, 4), '-', SUBSTRING(b_3.CODMES, 5, 2), '-01')
     left join catalog_lhcl_prod_bcp.bcp_ddv_rbmrbmper_fuenteinfocli_vu.hm_clientecemrbm as b_4
         on trim(a.CODCLAVEUNICOCLI)=trim(b_4.CODCLAVEUNICOCLI) and
         DATE_ADD(MONTH, -4, CAST(CONCAT(SUBSTRING(a.CODMES, 1, 4), '-', SUBSTRING(a.CODMES, 5, 2), '-01') AS DATE)) =CONCAT(SUBSTRING(b_4.CODMES, 1, 4), '-', SUBSTRING(b_4.CODMES, 5, 2), '-01')
     left join catalog_lhcl_prod_bcp.bcp_ddv_rbmrbmper_fuenteinfocli_vu.hm_clientecemrbm as b_5
         on trim(a.CODCLAVEUNICOCLI)=trim(b_5.CODCLAVEUNICOCLI) and
         DATE_ADD(MONTH, -5, CAST(CONCAT(SUBSTRING(a.CODMES, 1, 4), '-', SUBSTRING(a.CODMES, 5, 2), '-01') AS DATE)) =CONCAT(SUBSTRING(b_5.CODMES, 1, 4), '-', SUBSTRING(b_5.CODMES, 5, 2), '-01')
     left join catalog_lhcl_prod_bcp.bcp_ddv_rbmrbmper_fuenteinfocli_vu.hm_clientecemrbm as b_6
         on trim(a.CODCLAVEUNICOCLI)=trim(b_6.CODCLAVEUNICOCLI) and
         DATE_ADD(MONTH, -6, CAST(CONCAT(SUBSTRING(a.CODMES, 1, 4), '-', SUBSTRING(a.CODMES, 5, 2), '-01') AS DATE)) =CONCAT(SUBSTRING(b_6.CODMES, 1, 4), '-', SUBSTRING(b_6.CODMES, 5, 2), '-01')
     left join catalog_lhcl_prod_bcp.bcp_ddv_rbmrbmper_fuenteinfocli_vu.hm_clientecemrbm as b_F3
         on trim(a.CODCLAVEUNICOCLI)=trim(b_F3.CODCLAVEUNICOCLI) and
         DATE_ADD(MONTH, 3, CAST(CONCAT(SUBSTRING(a.CODMES, 1, 4), '-', SUBSTRING(a.CODMES, 5, 2), '-01') AS DATE)) =CONCAT(SUBSTRING(b_F3.CODMES, 1, 4), '-', SUBSTRING(b_F3.CODMES, 5, 2), '-01')
     left join catalog_lhcl_prod_bcp.bcp_ddv_rbmrbmper_fuenteinfocli_vu.hm_clientecemrbm as b_F6
         on trim(a.CODCLAVEUNICOCLI)=trim(b_F6.CODCLAVEUNICOCLI) and
         DATE_ADD(MONTH, 6, CAST(CONCAT(SUBSTRING(a.CODMES, 1, 4), '-', SUBSTRING(a.CODMES, 5, 2), '-01') AS DATE)) =CONCAT(SUBSTRING(b_F6.CODMES, 1, 4), '-', SUBSTRING(b_F6.CODMES, 5, 2), '-01')  
     ;
 """)

base_hip_6.persist(StorageLevel.MEMORY_AND_DISK).count()

dup = base_hip_6.groupBy("codclaveopecta").count().alias("count").filter(col("count")>1)
print_res(dup)

# Info salesforce

## Créditos hipotecarios cloud

In [0]:
from pyspark.sql.functions import date_format, concat, lit, lpad

sol_credit = spark.table("catalog_lhcl_prod_bcp.bcp_udv_int_vu.m_solicitudcreditohipotecario")

sol_credit_fin = sol_credit.filter(
    "desactividadhipotecario IN ('EMISION MEDIO DE PAGO', 'DESEMBOLSO')"
).select(
#    date_format(sol_credit.fecdesembolso, 'yyyyMM').alias('codmes'),
    sol_credit.desgradosostenibilidad,
    sol_credit.mtovalorcomercialinmueble,
    sol_credit.numscoreclititularappriesgo,
    sol_credit.destipcreditohipotecario,
    sol_credit.pctfinanciado,
    sol_credit.codsolicitudappriesgo,
    sol_credit.fecsolicitud,
    sol_credit.desactividadhipotecario,
    concat(lit('2'), lpad(sol_credit.codsolicitud, 10, '0')).alias('codsolicitud_nue')
).distinct()

sol_credit_fin.persist(StorageLevel.MEMORY_AND_DISK).count()

In [0]:
base_hip_6 = base_hip_6.join(
    sol_credit_fin.select(
        col("codsolicitud_nue").alias("codsolicitud"),
        col("pctfinanciado").alias("CSOL_LTV"),
        col("desgradosostenibilidad").alias("CSOL_GradoSostenibilidad"),
        col("numscoreclititularappriesgo").alias("CSOL_SC_RBM"),
        col("destipcreditohipotecario").alias("CSOL_tipcredito"),
        col("mtovalorcomercialinmueble").alias("CSOL_ValorComVivienda"),
        col("codsolicitudappriesgo"),
        col("fecsolicitud").alias("CSOL_FEC_SOL"),
        col("desactividadhipotecario").alias("ACT_HIP")
    ),
    on="codsolicitud",
    how='left'
)

base_hip_6.persist(StorageLevel.MEMORY_AND_DISK).count()

dup = base_hip_6.groupBy("codclaveopecta").count().alias("count").filter(col("count")>1)
print_res(dup)

In [0]:
from pyspark.sql.functions import col, exp

base_hip_6 = base_hip_6.withColumn("CSOL_XB_RBM", (174.25 - col("CSOL_SC_RBM")) / 57.708) \
                       .withColumn("CSOL_PD_RBM", 1 / (1 + exp(-col("CSOL_XB_RBM"))))

#Definicion de pasa pauta con excepciones
base_hip_6 = base_hip_6.withColumn(
    "FLG_PAUTA_VIV",
    F.when(
        (F.col("TIPO_FONDEO") != "FONDEO MVIV") | F.col("CODCLAVECIC").isNull(), None
    ).when(
        (F.col("CSOL_ValorComVivienda").between(61200, 218100)) & 
        (F.col("CSOL_GradoSostenibilidad") == "NINGUNO") & 
        (F.col("CSOL_SC_RBM").between(336, 385)), 1
    ).when(
        (F.col("CSOL_GradoSostenibilidad").isin("GRADO 1", "GRADO 2", "GRADO 3")) & 
        (F.col("CSOL_SC_RBM").between(336, 385)), 1
    ).when(
        F.col("CSOL_SC_RBM") >= 386, 1
    ).otherwise(0)
)

## Variables en fuente productiva cloud

In [0]:
rbm_nueva_fuente = spark.sql("""
    SELECT DISTINCT 
        codsolicitudcredito AS codsolicitud, 
        pctfinanciamientocredito AS NFT_LTV, 
        ctdanioplazoaprobado as NFT_PLAZO,
        mtoingresoconyuguesol as NFT_IngresoConyuge,
        mtoingresotitularsol  as NFT_IngresoTitular,
        desdistritoinmueble  as NFT_distritoinmueble, 
        desprovinciainmueble   as NFT_provinciainmueble,
        desdepartamentoinmueble as NFT_departamentoinmueble,
        desestadosolicitudcreditohipotecariorbm as NFT_estadosolicitud, 
        destipgrupocampaniasolicitudcreditocalibradorbm as NFT_grupocampania,
        codclavepartyemp as NFT_codclavepartyemp
    FROM catalog_lhcl_prod_bcp.bcp_ddv_rbmrbmper_solicitudch_vu.hm_solicitudcreditohipotecariorbm
    WHERE fecactualizacionregistro = (
        SELECT MAX(fecactualizacionregistro)
        FROM catalog_lhcl_prod_bcp.bcp_ddv_rbmrbmper_solicitudch_vu.hm_solicitudcreditohipotecariorbm
    )
""")

rbm_nueva_fuente.persist(StorageLevel.MEMORY_AND_DISK).count()

dup = rbm_nueva_fuente.groupBy("codsolicitud").count().alias("count").filter(col("count")>1)
print_res(dup)

In [0]:
# Pegando variables demográficas de la nueva fuente
base_hip_7 = base_hip_6.join(
    rbm_nueva_fuente.selectExpr("codsolicitud","NFT_LTV","NFT_PLAZO", "NFT_IngresoConyuge", "NFT_IngresoTitular",  "NFT_distritoinmueble",  "NFT_provinciainmueble", "NFT_departamentoinmueble", "NFT_estadosolicitud", "NFT_grupocampania"),
    on = ["codsolicitud"],
    how = 'left'
)

base_hip_7.persist(StorageLevel.MEMORY_AND_DISK).count()

dup = base_hip_7.groupBy("codclaveopecta").count().alias("count").filter(col("count")>1)
print_res(dup)

## Suite Cloud

In [0]:
# Titular
suite_titular = spark.sql("""
    SELECT a.*
    FROM catalog_lhcl_prod_bcp.bcp_ddv_rbmrbmper_reportlogevaluacion_vu.hd_personasolicitudevaluacioncreditohipoticariorbmper a
    INNER JOIN (
        SELECT numsecuencialsolicitudcreditohipotecario, MAX(fecrutina) AS max_fecrutina
        FROM catalog_lhcl_prod_bcp.bcp_ddv_rbmrbmper_reportlogevaluacion_vu.hd_personasolicitudevaluacioncreditohipoticariorbmper 
        WHERE destiporelacionadoevaluacion = 'TITULAR'
        GROUP BY numsecuencialsolicitudcreditohipotecario
    ) b
    ON a.numsecuencialsolicitudcreditohipotecario = b.numsecuencialsolicitudcreditohipotecario AND a.fecrutina = b.max_fecrutina
    WHERE a.destiporelacionadoevaluacion = 'TITULAR'
""")

suite_titular.persist(StorageLevel.MEMORY_AND_DISK).count()

# Conyuge
suite_conyuge = spark.sql("""
    SELECT a.*
    FROM catalog_lhcl_prod_bcp.bcp_ddv_rbmrbmper_reportlogevaluacion_vu.hd_personasolicitudevaluacioncreditohipoticariorbmper a
    INNER JOIN (
        SELECT numsecuencialsolicitudcreditohipotecario, MAX(fecrutina) AS max_fecrutina
        FROM catalog_lhcl_prod_bcp.bcp_ddv_rbmrbmper_reportlogevaluacion_vu.hd_personasolicitudevaluacioncreditohipoticariorbmper 
        WHERE destiporelacionadoevaluacion = 'CONYUGE'
        GROUP BY numsecuencialsolicitudcreditohipotecario
    ) b
    ON a.numsecuencialsolicitudcreditohipotecario = b.numsecuencialsolicitudcreditohipotecario AND a.fecrutina = b.max_fecrutina
    WHERE a.destiporelacionadoevaluacion = 'CONYUGE'
""")

suite_conyuge.persist(StorageLevel.MEMORY_AND_DISK).count()

In [0]:
ver = spark.table('catalog_lhcl_prod_bcp.bcp_ddv_rbmrbmper_reportlogevaluacion_vu.hd_personasolicitudevaluacioncreditohipoticariorbmper')

In [0]:
base_hip_8 = base_hip_7.join(
    suite_titular.selectExpr(
        "tipestadocivil as SUITE_Estcivil",
        "fecnacimiento as SUITE_Fecnacimiento",
        "fecingresolaboral  as SUITE_Fecingresolaboral",
        "numsecuencialsolicitudcreditohipotecario as codsolicitudappriesgo",
        "desdistrito as SUITE_desdistrito","fecdia as SUITE_fecdia",
        "desprovincia as SUITE_desprovincia",
        "desdepartamento as SUITE_desdepartamento",
        "codinternocomputacionalempleador",
        "codclaveunicocliempleador",
        "codclavepartycliempleador"
    ),
    on = ["codsolicitudappriesgo"],
    how = 'left'
)

base_hip_8.persist(StorageLevel.MEMORY_AND_DISK).count()

dup = base_hip_8.groupBy("codclaveopecta").count().alias("count").filter(col("count")>1)
print_res(dup)

## Excepciones

In [0]:
# Excepciones historicas
base_excepciones_hist = spark.sql("select distinct Nombre,Tipo_excepcion from catalog_lhcl_prod_bcp.bcp_edv_fabseg.T45988_HM_EXCEPCIONES_HIPOTECARIO where ID_Excepcion is not null and Nombre is not null")

# Excepciones Marzo
base_excepciones_actual = spark.sql("select distinct Nombre,`Tipo de excepci�n`as Tipo_excepcion from catalog_lhcl_prod_bcp.bcp_edv_fabseg.T45988_HM_EXCEPCIONES_HIPOTECARIO_202503 where `ID Excepci�n` is not null and Nombre is not null")

base_excepciones = base_excepciones_hist.union(base_excepciones_actual)

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import regexp_replace

base_excepciones_unico = spark.sql("""
SELECT 
    nombre, 
    SUM(CASE WHEN Tipo_excepcion = 'Perfil del Cliente' THEN 1 ELSE 0 END) AS flg_perfil,
    SUM(CASE WHEN Tipo_excepcion = 'Score' THEN 1 ELSE 0 END) AS flG_score,
    SUM(CASE WHEN UPPER(Tipo_excepcion) LIKE 'CONDICION%' THEN 1 ELSE 0 END) AS flg_CondicionCredito,
    SUM(CASE WHEN Tipo_excepcion = 'Plan de Ahorro' THEN 1 ELSE 0 END) AS flg_PlanAhorro,
    SUM(CASE WHEN Tipo_excepcion = 'Ingresos' THEN 1 ELSE 0 END) AS flg_Ingresos,
    SUM(CASE WHEN Tipo_excepcion = 'Inmobiliario y legal' THEN 1 ELSE 0 END) AS flg_Inmobiliario,
    SUM(CASE WHEN Tipo_excepcion = 'ISFP' THEN 1 ELSE 0 END) AS flg_ISFP,
    SUM(CASE WHEN Tipo_excepcion = 'Seguros' THEN 1 ELSE 0 END) AS flg_Seguros,
    COUNT(1) AS n_exc
FROM (
    SELECT DISTINCT nombre, Tipo_excepcion 
    FROM catalog_lhcl_prod_bcp.bcp_edv_fabseg.T45988_HM_EXCEPCIONES_HIPOTECARIO_HIST_202504
) AS t
GROUP BY nombre
""")
base_excepciones_unico = base_excepciones_unico.withColumn("CODSOLICITUD", regexp_replace(base_excepciones_unico["nombre"], "^N", ""))

In [0]:
base_hip_8 = base_hip_8.join(
    base_excepciones_unico,
    on = ["codsolicitud"],
    how = 'left'
)

base_hip_8.persist(StorageLevel.MEMORY_AND_DISK).count()

dup = base_hip_8.groupBy("codclaveopecta").count().alias("count").filter(col("count")>1)
print_res(dup)

In [0]:
from pyspark.sql.functions import months_between

base_hip_8 = base_hip_8.withColumn("SUITE_AntLaboralMeses", months_between(base_hip_8["SUITE_fecdia"], base_hip_8["SUITE_Fecingresolaboral"]))
base_hip_8 = base_hip_8.withColumn("SUITE_Fecnacimiento", base_hip_8["SUITE_Fecnacimiento"].cast("date"))
base_hip_8 = base_hip_8.withColumn("SUITE_Edad", months_between(base_hip_8["CSOL_FEC_SOL"], base_hip_8["SUITE_Fecnacimiento"]) / 12)

In [0]:
base_hip_8 = base_hip_8.dropDuplicates(subset=['CODCLAVEOPECTA'])
base_hip_8.createOrReplaceTempView('base_hip_8')

spark.sql('''
    CREATE OR REPLACE TABLE
        catalog_lhcl_prod_bcp.bcp_edv_fabseg.T39290_univ_pre_driver_app_hip
    USING DELTA LOCATION 
        'abfss://bcp-edv-fabseg@adlscu1lhclbackp05.dfs.core.windows.net/T39290/Seguimiento/02_Applicant_Hipotecario/univ_pre_driver_app_hip'
    AS SELECT * FROM base_hip_8
    '''
)